In [ ]:
import json
import os
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
SEED = 1001
DROP_APPROACHES = [
    'BinSeg Clinear',
]
EXPECTED_NUMBER_APPROACHES = (3 * 10) + 1 - len(DROP_APPROACHES)

In [4]:
def get_df_for_dataset(database_name: str, dataset_name: str) -> pd.DataFrame:
    def get_df_for_dataset_inner(database_name: str, dataset_name: str) -> pd.DataFrame:
        directory_path = f"outputs/report/seed={SEED}/{database_name}/{dataset_name}"

        print(f"Reading from {directory_path}")

        metrics_dfs = []
        
        for root, dirs, files in os.walk(directory_path):
            for filename in files:
                if filename.endswith('.json'):
                    file_path = os.path.join(root, filename)
                    
                    try:
                        with open(file_path, 'r') as file:
                            data = json.load(file)
                            metrics_dfs.append(
                                pd.DataFrame({
                                    'cut_point_model': [data['cut_point_model']],
                                    'cut_point_method': [data['cut_point_method']],
                                    'cut_point_approach': [data['cut_point_approach']],
                                    'cut_point_perc': [data['cut_point_perc']],
                                    'cut_duration': [data['cut_duration']],
                                    'cut_duration_minutes': [data['cut_duration'] / 60],
                                    'tuner_duration': [data['tuner_duration']],
                                    'tuner_duration_minutes': [data['tuner_duration'] / 60],
                                    'retrain_duration': [data['retrain_duration']],
                                    'retrain_duration_minutes': [data['retrain_duration'] / 60],
                                    'total_duration': [data['total_duration']],
                                    'total_duration_minutes': [data['total_duration'] / 60],
                                    'Avg_MAPE': [data['error_results']['Avg_MAPE']],
                                    'Avg_MAE': [data['error_results']['Avg_MAE']], 
                                    'Avg_MSE': [data['error_results']['Avg_MSE']], 
                                    'Avg_RMSE': [data['error_results']['Avg_RMSE']], 
                                    'Avg_R2':  [data['error_results']['Avg_R2']],
                                    'Avg_WAPE':  [data['error_results']['Avg_WAPE']],
                                }))
                    except KeyError as e:
                        print(f"Error in {file_path}: {e}")
                        continue
                    
        metrics_df = pd.concat(metrics_dfs, axis=0)
    
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Rbf', 'RBF')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace(' Ar', ' AR')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Bin_Seg', 'BinSeg')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Bottom_Up', 'BottomUp')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Fixed_Perc Fixed_Cut_0.0', 'Full')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('Fixed_Perc Fixed_Cut_', 'Fixed Cut ')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.1', '10%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.2', '20%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.3', '30%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.4', '40%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.5', '50%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.6', '60%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.7', '70%')
        metrics_df['cut_point_approach'] = metrics_df['cut_point_approach'].str.replace('0.8', '80%')

        metrics_df = metrics_df[~metrics_df['cut_point_approach'].str.endswith(tuple(DROP_APPROACHES))]

        return metrics_df

    metrics_df = get_df_for_dataset_inner(
        database_name, dataset_name
    )

    metrics_cols = metrics_df.columns.tolist()
    metrics_df['database'] = [database_name] * len(metrics_df)
    metrics_df['dataset'] = [dataset_name] * len(metrics_df)
    metrics_df = metrics_df[['database', 'dataset'] + metrics_cols]

    return metrics_df.reset_index(drop=True)

In [5]:
datasets = []

# [UCI] APPLIANCES_ENERGY

In [6]:
uci_appliances_energy = get_df_for_dataset('UCI', 'APPLIANCES_ENERGY')
uci_appliances_energy_name = 'UCI APPLIANCES_ENERGY'
assert not uci_appliances_energy['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
assert len(uci_appliances_energy) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(uci_appliances_energy)}"

datasets.append(uci_appliances_energy)

uci_appliances_energy.sort_values(by='Avg_RMSE')

Reading from outputs/report/seed=1001/UCI/APPLIANCES_ENERGY


,database,dataset,cut_point_model,cut_point_method,cut_point_approach,cut_point_perc,cut_duration,cut_duration_minutes,tuner_duration,tuner_duration_minutes,retrain_duration,retrain_duration_minutes,total_duration,total_duration_minutes,Avg_MAPE,Avg_MAE,Avg_MSE,Avg_RMSE,Avg_R2,Avg_WAPE
9,UCI,APPLIANCES_ENERGY,Bottom_Up,Clinear,BottomUp Clinear,1.330124,0.050418,8.402983e-04,1302.820572,21.713676,11.294675,0.188245,1314.165665,21.902761,1.189856e+12,1.933883,9.697113,2.591208,0.823831,0.013052
8,UCI,APPLIANCES_ENERGY,Bottom_Up,Rank,BottomUp Rank,53.870028,0.124205,2.070085e-03,787.144698,13.119078,5.769621,0.096160,793.038524,13.217309,1.500803e+12,2.048562,10.763632,2.718472,0.807064,0.013826
17,UCI,APPLIANCES_ENERGY,Window,L1,Window L1,53.585001,0.140916,2.348602e-03,807.124324,13.452072,5.077443,0.084624,812.342683,13.539045,1.777477e+12,2.039548,10.275543,2.737941,0.807361,0.013766
14,UCI,APPLIANCES_ENERGY,Window,AR,Window AR,60.583988,0.108044,1.800736e-03,636.343760,10.605729,4.731203,0.078853,641.183007,10.686383,1.481241e+12,2.053318,10.877731,2.744123,0.822312,0.013858
24,UCI,APPLIANCES_ENERGY,Bin_Seg,AR,BinSeg AR,38.985305,3.979118,6.631863e-02,1067.972468,17.799541,9.217237,0.153621,1081.168823,18.019480,1.210166e+12,2.067946,10.275667,2.770830,0.794089,0.013957
26,UCI,APPLIANCES_ENERGY,Bin_Seg,Mahalanobis,BinSeg Mahalanobis,61.850773,252.155165,4.202586e+00,670.868701,11.181145,4.690048,0.078167,927.713914,15.461899,1.596949e+12,2.131711,11.939911,2.815712,0.820692,0.014388
2,UCI,APPLIANCES_ENERGY,Bottom_Up,L2,BottomUp L2,51.748163,0.057734,9.622296e-04,823.077386,13.717956,5.679653,0.094661,828.814773,13.813580,1.683439e+12,2.124875,11.301123,2.824988,0.798128,0.014341
6,UCI,APPLIANCES_ENERGY,Bottom_Up,Mahalanobis,BottomUp Mahalanobis,23.498860,0.817191,1.361985e-02,1150.236234,19.170604,7.474998,0.124583,1158.528423,19.308807,1.295651e+12,2.167381,10.982065,2.858909,0.793276,0.014628
4,UCI,APPLIANCES_ENERGY,Bottom_Up,AR,BottomUp AR,53.299975,0.078179,1.302981e-03,707.539017,11.792317,5.685189,0.094753,713.302385,11.888373,1.683133e+12,2.138622,11.194739,2.860968,0.789339,0.014434
22,UCI,APPLIANCES_ENERGY,Bin_Seg,L2,BinSeg L2,51.779833,0.448889,7.481480e-03,837.642100,13.960702,6.044348,0.100739,844.135337,14.068922,1.648034e+12,2.164705,11.789310,2.872262,0.798515,0.014610


# [UCI] METRO_TRAFFIC

In [7]:
uci_metro_traffic = get_df_for_dataset('UCI', 'METRO_TRAFFIC')
uci_metro_traffic_name = 'UCI METRO_TRAFFIC'
assert not uci_metro_traffic['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
assert len(uci_metro_traffic) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(uci_metro_traffic)}"

datasets.append(uci_metro_traffic)

uci_metro_traffic.sort_values(by='Avg_RMSE')

Reading from outputs/report/seed=1001/UCI/METRO_TRAFFIC


,database,dataset,cut_point_model,cut_point_method,cut_point_approach,cut_point_perc,cut_duration,cut_duration_minutes,tuner_duration,tuner_duration_minutes,retrain_duration,retrain_duration_minutes,total_duration,total_duration_minutes,Avg_MAPE,Avg_MAE,Avg_MSE,Avg_RMSE,Avg_R2,Avg_WAPE
1,UCI,METRO_TRAFFIC,Bottom_Up,Linear,BottomUp Linear,70.300547,0.198101,3.301684e-03,727.565145,12.126086,14.429776,0.240496,742.193022,12.369884,9.368224e+13,10.170932,331.754795,12.457040,0.393282,0.093086
25,UCI,METRO_TRAFFIC,Bin_Seg,Normal,BinSeg Normal,70.300547,2.756494,4.594156e-02,728.135074,12.135585,14.067820,0.234464,744.959388,12.415990,9.368224e+13,10.170932,331.754795,12.457040,0.393282,0.093086
21,UCI,METRO_TRAFFIC,Bin_Seg,Linear,BinSeg Linear,70.300547,4.371435,7.285725e-02,689.778681,11.496311,13.348889,0.222481,707.499005,11.791650,9.368224e+13,10.170932,331.754795,12.457040,0.393282,0.093086
5,UCI,METRO_TRAFFIC,Bottom_Up,Normal,BottomUp Normal,70.300547,0.186291,3.104854e-03,709.761484,11.829358,13.747296,0.229122,723.695071,12.061585,9.368224e+13,10.170932,331.754795,12.457040,0.393282,0.093086
17,UCI,METRO_TRAFFIC,Window,L1,Window L1,78.404170,0.307796,5.129933e-03,776.766402,12.946107,13.692857,0.228214,790.767055,13.179451,9.439665e+13,10.338564,324.540313,12.540456,0.383618,0.094621
13,UCI,METRO_TRAFFIC,Window,RBF,Window RBF,78.404170,207.259021,3.454317e+00,704.957848,11.749297,12.929344,0.215489,925.146213,15.419104,9.439665e+13,10.338564,324.540313,12.540456,0.383618,0.094621
29,UCI,METRO_TRAFFIC,Fixed_Perc,Fixed_Cut_0.0,Full,0.000000,0.000007,1.152356e-07,2552.992444,42.549874,71.293405,1.188223,2624.285856,43.738098,1.841010e+15,10.534125,325.852810,12.654029,-2.453238,0.096410
16,UCI,METRO_TRAFFIC,Window,Mahalanobis,Window Mahalanobis,64.543734,11.637247,1.939541e-01,862.164302,14.369405,7.614407,0.126907,881.415956,14.690266,1.813163e+14,10.462652,337.180392,12.868888,0.478499,0.095756
12,UCI,METRO_TRAFFIC,Window,L2,Window L2,64.543734,0.199784,3.329734e-03,901.981846,15.033031,8.324569,0.138743,910.506199,15.175103,1.813163e+14,10.462652,337.180392,12.868888,0.478499,0.095756
2,UCI,METRO_TRAFFIC,Bottom_Up,L2,BottomUp L2,64.530768,0.119341,1.989015e-03,764.392951,12.739883,7.569201,0.126153,772.081493,12.868025,1.843881e+14,10.468469,336.725484,12.931362,0.472238,0.095809


# [UCI] PRSA_BEIJING

In [8]:
uci_prsa_beijing = get_df_for_dataset('UCI', 'PRSA_BEIJING')
uci_prsa_beijing_name = 'UCI PRSA_BEIJING'
assert not uci_prsa_beijing['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
assert len(uci_prsa_beijing) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(uci_prsa_beijing)}"

datasets.append(uci_prsa_beijing)

uci_prsa_beijing.sort_values(by='Avg_RMSE')

Reading from outputs/report/seed=1001/UCI/PRSA_BEIJING


,database,dataset,cut_point_model,cut_point_method,cut_point_approach,cut_point_perc,cut_duration,cut_duration_minutes,tuner_duration,tuner_duration_minutes,retrain_duration,retrain_duration_minutes,total_duration,total_duration_minutes,Avg_MAPE,Avg_MAE,Avg_MSE,Avg_RMSE,Avg_R2,Avg_WAPE
16,UCI,PRSA_BEIJING,Window,Mahalanobis,Window Mahalanobis,0.199663,5.990686,9.984477e-02,1868.422061,31.140368,22.860102,0.381002,1897.272849,31.621214,2.591731e+14,4.153104,118.391732,6.720462,0.671715,0.023425
3,UCI,PRSA_BEIJING,Bottom_Up,RBF,BottomUp RBF,8.186200,109.356600,1.822610e+00,1593.805372,26.563423,22.033571,0.367226,1725.195543,28.753259,3.402254e+14,4.147777,113.900162,6.741526,0.662415,0.023395
7,UCI,PRSA_BEIJING,Bottom_Up,L1,BottomUp L1,8.186200,0.188692,3.144864e-03,1639.362887,27.322715,26.914572,0.448576,1666.466151,27.774436,3.402254e+14,4.147777,113.900162,6.741526,0.662415,0.023395
22,UCI,PRSA_BEIJING,Bin_Seg,L2,BinSeg L2,35.368949,2.065835,3.443058e-02,1174.901798,19.581697,13.615906,0.226932,1190.583539,19.843059,4.363414e+14,4.301501,115.738078,6.839334,0.628829,0.024262
20,UCI,PRSA_BEIJING,Bin_Seg,Cosine,BinSeg Cosine,35.368949,2136.977374,3.561629e+01,1155.429338,19.257156,12.861606,0.214360,3305.268318,55.087805,4.363414e+14,4.301501,115.738078,6.839334,0.628829,0.024262
18,UCI,PRSA_BEIJING,Window,Rank,Window Rank,0.213925,0.195136,3.252268e-03,1942.526512,32.375442,26.156080,0.435935,1968.877728,32.814629,3.200049e+14,4.236825,118.465932,6.839464,0.648220,0.023898
13,UCI,PRSA_BEIJING,Window,RBF,Window RBF,21.920192,129.083524,2.151392e+00,1361.913061,22.698551,22.085045,0.368084,1513.081630,25.218027,3.374832e+14,4.285121,113.436168,6.848528,0.640798,0.024170
26,UCI,PRSA_BEIJING,Bin_Seg,Mahalanobis,BinSeg Mahalanobis,5.505006,2835.531877,4.725886e+01,1536.575929,25.609599,14.260019,0.237667,4386.367825,73.106130,3.031254e+14,4.386233,123.365354,6.869200,0.685442,0.024740
28,UCI,PRSA_BEIJING,Bin_Seg,Rank,BinSeg Rank,5.505006,2.833622,4.722703e-02,1624.646987,27.077450,15.246511,0.254109,1642.727120,27.378785,3.031254e+14,4.386233,123.365354,6.869200,0.685442,0.024740
27,UCI,PRSA_BEIJING,Bin_Seg,L1,BinSeg L1,5.989903,13.926591,2.321099e-01,1516.222354,25.270373,13.853200,0.230887,1544.002145,25.733369,3.331807e+14,4.731535,121.906798,6.991363,0.619171,0.026688


# [UCI] AIR_QUALITY

In [9]:
uci_air_quality = get_df_for_dataset('UCI', 'AIR_QUALITY')
uci_air_quality_name = 'UCI AIR_QUALITY'
assert not uci_air_quality['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
assert len(uci_air_quality) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(uci_air_quality)}"

datasets.append(uci_air_quality)

uci_air_quality.sort_values(by='Avg_RMSE')

Reading from outputs/report/seed=1001/UCI/AIR_QUALITY
Error in outputs/report/seed=1001/UCI/AIR_QUALITY/Bin_Seg/Clinear/2025-03-05_09-58-48/report.json: 'tuner_duration'


,database,dataset,cut_point_model,cut_point_method,cut_point_approach,cut_point_perc,cut_duration,cut_duration_minutes,tuner_duration,tuner_duration_minutes,retrain_duration,retrain_duration_minutes,total_duration,total_duration_minutes,Avg_MAPE,Avg_MAE,Avg_MSE,Avg_RMSE,Avg_R2,Avg_WAPE
10,UCI,AIR_QUALITY,Window,Cosine,Window Cosine,10.754843,0.215372,3.589535e-03,249.990173,4.166503,4.018255,0.066971,254.223800,4.237063,5.123238e+12,28.261199,4978.753985,39.882094,0.165600,0.330999
29,UCI,AIR_QUALITY,Fixed_Perc,Fixed_Cut_0.0,Full,0.000000,0.000006,9.934107e-08,331.911947,5.531866,4.017133,0.066952,335.929086,5.598818,5.514348e+12,29.172569,5552.604935,41.671120,0.093280,0.341673
28,UCI,AIR_QUALITY,Bin_Seg,Rank,BinSeg Rank,55.511022,0.176376,2.939602e-03,169.092951,2.818216,3.842953,0.064049,173.112280,2.885205,4.308143e+12,32.863112,5992.969973,42.260849,0.231873,0.384898
4,UCI,AIR_QUALITY,Bottom_Up,AR,BottomUp AR,55.778223,0.035727,5.954464e-04,193.986755,3.233113,3.727774,0.062130,197.750256,3.295838,4.438382e+12,33.320460,6113.528316,42.490627,0.236175,0.390254
21,UCI,AIR_QUALITY,Bin_Seg,Linear,BinSeg Linear,52.371409,0.482256,8.037599e-03,208.458791,3.474313,3.338716,0.055645,212.279763,3.537996,4.243376e+12,33.545168,6211.587301,42.891739,0.243044,0.392886
0,UCI,AIR_QUALITY,Bottom_Up,Cosine,BottomUp Cosine,54.976620,0.289178,4.819636e-03,161.248114,2.687469,6.842112,0.114035,168.379405,2.806323,3.485876e+12,32.426023,6373.930598,43.012693,0.354242,0.379778
8,UCI,AIR_QUALITY,Bottom_Up,Rank,BottomUp Rank,56.179025,0.069745,1.162414e-03,187.805723,3.130095,4.397272,0.073288,192.272740,3.204546,4.678243e+12,33.109409,6546.864528,43.259411,0.295374,0.387782
9,UCI,AIR_QUALITY,Bottom_Up,Clinear,BottomUp Clinear,56.513026,0.022433,3.738801e-04,205.207949,3.420132,2.337398,0.038957,207.567780,3.459463,4.265050e+12,34.208883,6337.787004,43.368133,0.174926,0.400659
5,UCI,AIR_QUALITY,Bottom_Up,Normal,BottomUp Normal,56.513026,0.045893,7.648865e-04,212.491173,3.541520,2.788454,0.046474,215.325520,3.588759,4.265050e+12,34.208883,6337.787004,43.368133,0.174926,0.400659
25,UCI,AIR_QUALITY,Bin_Seg,Normal,BinSeg Normal,55.911824,0.245499,4.091648e-03,178.750708,2.979178,14.656704,0.244278,193.652911,3.227549,4.735708e+12,33.872056,6878.822839,44.187661,0.248341,0.396714


# [INMET] SAOPAULO_SP

In [10]:
# inmet_saopaulo_sp = get_df_for_dataset('INMET', 'SAOPAULO_SP')
# inmet_saopaulo_sp_name = 'INMET SAOPAULO_SP'
# assert not inmet_saopaulo_sp['cut_point_approach'].duplicated().any(), "Duplicate values found in column 'cut_point_approach'"
# assert len(inmet_saopaulo_sp) == EXPECTED_NUMBER_APPROACHES, f"Expected {EXPECTED_NUMBER_APPROACHES} lines but got {len(inmet_saopaulo_sp)}"

# datasets.append(inmet_saopaulo_sp)

# inmet_saopaulo_sp.sort_values(by='Avg_RMSE')

# Rank

In [11]:
def compute_arr(df, acc_d, algorithm_p, algorithm_q, dataset, metric):
    df_d = df[df['dataset_name'] == dataset]

    d_ap = df_d[df_d["cut_point_approach"] == algorithm_p]
    d_aq = df_d[df_d["cut_point_approach"] == algorithm_q]

    assert len(d_ap) == 1, f"Expected 1 line for approach {algorithm_p} but got {len(d_ap)}"
    assert len(d_aq) == 1, f"Expected 1 line for approach {algorithm_q} but got {len(d_aq)}"

    sr_ap = 1 / d_ap[metric].iloc[0]
    sr_aq = 1 / d_aq[metric].iloc[0]

    t_ap = d_ap["total_duration"].iloc[0]
    t_aq = d_aq["total_duration"].iloc[0]

    sr_ratio = sr_ap / sr_aq if sr_aq != 0 else np.nan  # Avoid division by zero
    time_ratio = np.log(t_ap / t_aq) if t_aq != 0 else np.nan  # Avoid division by zero

    arr = sr_ratio / (1 + acc_d * time_ratio)
    
    return arr


def compute_arr_ranking(df, acc_d, metric):
    algorithms = df["cut_point_approach"].unique()
    datasets = df['dataset_name'].unique()
    arr_df = pd.DataFrame({
        'dataset': [],
        'ap': [],
        'aq': [],
        'arr': [],
    })

    for dataset in datasets:
        for ap in algorithms:
            for aq in algorithms:
                if ap == aq:
                    continue

                arr_df = pd.concat([arr_df, pd.DataFrame({
                    'dataset': [dataset],
                    'ap': [ap],
                    'aq': [aq],
                    'arr': [compute_arr(df, acc_d, ap, aq, dataset, metric)],
                })])

    grouped = arr_df.groupby(['ap', 'aq'])['arr'].apply(lambda x: np.prod(x) ** (1 / len(x))).reset_index()

    ranking_df = grouped.groupby('ap')['arr'].mean().reset_index()
    ranking_df['rank'] = ranking_df['arr'].rank(ascending=False, method='max')

    return ranking_df.sort_values(by='rank')

In [12]:
metrics_df = pd.concat(datasets, axis=0)
metrics_df = metrics_df[~metrics_df['cut_point_approach'].isin(DROP_APPROACHES)]
metrics_df['dataset_name'] = metrics_df['database'] + ' ' + metrics_df['dataset']

## WAPE

In [13]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.05, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
18,BottomUp Rank,1.111680,1.0
22,Window Cosine,1.089084,2.0
23,Window L1,1.086244,3.0
17,BottomUp RBF,1.083082,4.0
8,BinSeg Rank,1.082111,5.0
3,BinSeg L2,1.079035,6.0
19,Full,1.076924,7.0
9,BottomUp AR,1.068483,8.0
13,BottomUp L2,1.061815,9.0
28,Window RBF,1.060284,10.0


In [14]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.1, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
18,BottomUp Rank,1.103186,1.0
23,Window L1,1.090113,2.0
16,BottomUp Normal,1.087881,3.0
17,BottomUp RBF,1.078612,4.0
8,BinSeg Rank,1.074206,5.0
22,Window Cosine,1.072943,6.0
3,BinSeg L2,1.071885,7.0
9,BottomUp AR,1.069444,8.0
13,BottomUp L2,1.063852,9.0
28,Window RBF,1.055176,10.0


In [15]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.25, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
10,BottomUp Clinear,1.535054,1.0
16,BottomUp Normal,1.281039,2.0
4,BinSeg Linear,1.259544,3.0
6,BinSeg Normal,1.223476,4.0
20,Window AR,1.178405,5.0
14,BottomUp Linear,1.167269,6.0
23,Window L1,1.138754,7.0
13,BottomUp L2,1.107394,8.0
18,BottomUp Rank,1.096938,9.0
9,BottomUp AR,1.095681,10.0


In [16]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.5, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
14,BottomUp Linear,2.305140,1.0
16,BottomUp Normal,1.802774,2.0
4,BinSeg Linear,1.672211,3.0
6,BinSeg Normal,1.616665,4.0
10,BottomUp Clinear,1.615245,5.0
20,Window AR,1.484256,6.0
25,Window Linear,1.366215,7.0
27,Window Normal,1.313270,8.0
26,Window Mahalanobis,1.233366,9.0
24,Window L2,1.164999,10.0


In [17]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.75, metric='Avg_WAPE')
arr_ranking_df

,ap,arr,rank
14,BottomUp Linear,2.035518,1.0
20,Window AR,1.950786,2.0
25,Window Linear,1.533254,3.0
26,Window Mahalanobis,1.408497,4.0
4,BinSeg Linear,1.406235,5.0
29,Window Rank,1.382895,6.0
27,Window Normal,1.382595,7.0
6,BinSeg Normal,1.337680,8.0
16,BottomUp Normal,1.297289,9.0
11,BottomUp Cosine,1.184724,10.0


## MSE

In [18]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.05, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
22,Window Cosine,1.185594,1.0
23,Window L1,1.160919,2.0
11,BottomUp Cosine,1.159714,3.0
18,BottomUp Rank,1.157973,4.0
19,Full,1.146060,5.0
9,BottomUp AR,1.137887,6.0
8,BinSeg Rank,1.136523,7.0
12,BottomUp L1,1.130256,8.0
17,BottomUp RBF,1.128186,9.0
3,BinSeg L2,1.128158,10.0


In [19]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.1, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
22,Window Cosine,1.166972,1.0
23,Window L1,1.163905,2.0
18,BottomUp Rank,1.148043,3.0
11,BottomUp Cosine,1.148015,4.0
16,BottomUp Normal,1.141836,5.0
9,BottomUp AR,1.137797,6.0
8,BinSeg Rank,1.127155,7.0
12,BottomUp L1,1.123387,8.0
17,BottomUp RBF,1.122455,9.0
3,BinSeg L2,1.119624,10.0


In [20]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.25, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
10,BottomUp Clinear,1.580206,1.0
16,BottomUp Normal,1.337215,2.0
4,BinSeg Linear,1.262008,3.0
6,BinSeg Normal,1.256880,4.0
23,Window L1,1.210817,5.0
9,BottomUp AR,1.161596,6.0
13,BottomUp L2,1.158888,7.0
20,Window AR,1.150482,8.0
18,BottomUp Rank,1.137982,9.0
11,BottomUp Cosine,1.130307,10.0


In [21]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.5, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
14,BottomUp Linear,1.901805,1.0
16,BottomUp Normal,1.844286,2.0
4,BinSeg Linear,1.654290,3.0
6,BinSeg Normal,1.638922,4.0
10,BottomUp Clinear,1.630500,5.0
20,Window AR,1.445910,6.0
27,Window Normal,1.333173,7.0
23,Window L1,1.226342,8.0
9,BottomUp AR,1.204292,9.0
13,BottomUp L2,1.163681,10.0


In [22]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.75, metric='Avg_MSE')
arr_ranking_df

,ap,arr,rank
20,Window AR,1.862814,1.0
14,BottomUp Linear,1.655771,2.0
4,BinSeg Linear,1.473055,3.0
6,BinSeg Normal,1.457098,4.0
16,BottomUp Normal,1.410385,5.0
27,Window Normal,1.406078,6.0
25,Window Linear,1.306486,7.0
11,BottomUp Cosine,1.282130,8.0
23,Window L1,1.258491,9.0
13,BottomUp L2,1.190509,10.0


## RMSE

In [23]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.05, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
18,BottomUp Rank,1.091054,1.0
23,Window L1,1.086170,2.0
17,BottomUp RBF,1.075395,3.0
8,BinSeg Rank,1.071764,4.0
13,BottomUp L2,1.062966,5.0
3,BinSeg L2,1.062006,6.0
22,Window Cosine,1.060904,7.0
19,Full,1.058318,8.0
16,BottomUp Normal,1.057127,9.0
28,Window RBF,1.053563,10.0


In [24]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.1, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
16,BottomUp Normal,1.091408,1.0
23,Window L1,1.090225,2.0
18,BottomUp Rank,1.082885,3.0
17,BottomUp RBF,1.071126,4.0
10,BottomUp Clinear,1.065851,5.0
13,BottomUp L2,1.065185,6.0
8,BinSeg Rank,1.064101,7.0
3,BinSeg L2,1.055139,8.0
9,BottomUp AR,1.054388,9.0
28,Window RBF,1.048661,10.0


In [25]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.25, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
10,BottomUp Clinear,1.607970,1.0
16,BottomUp Normal,1.286076,2.0
4,BinSeg Linear,1.253754,3.0
6,BinSeg Normal,1.224563,4.0
14,BottomUp Linear,1.188427,5.0
20,Window AR,1.182486,6.0
23,Window L1,1.139424,7.0
13,BottomUp L2,1.109311,8.0
0,BinSeg AR,1.100221,9.0
9,BottomUp AR,1.080750,10.0


In [26]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.5, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
14,BottomUp Linear,2.343767,1.0
16,BottomUp Normal,1.808808,2.0
10,BottomUp Clinear,1.694450,3.0
4,BinSeg Linear,1.660849,4.0
6,BinSeg Normal,1.614577,5.0
20,Window AR,1.488533,6.0
25,Window Linear,1.360134,7.0
27,Window Normal,1.337731,8.0
26,Window Mahalanobis,1.236129,9.0
24,Window L2,1.162262,10.0


In [27]:
arr_ranking_df = compute_arr_ranking(metrics_df, acc_d=0.75, metric='Avg_RMSE')
arr_ranking_df

,ap,arr,rank
14,BottomUp Linear,2.088130,1.0
20,Window AR,1.961781,2.0
25,Window Linear,1.525298,3.0
27,Window Normal,1.411639,4.0
26,Window Mahalanobis,1.406836,5.0
4,BinSeg Linear,1.386189,6.0
29,Window Rank,1.380682,7.0
6,BinSeg Normal,1.328238,8.0
16,BottomUp Normal,1.289953,9.0
23,Window L1,1.185365,10.0
